#                                            **Self Organising Maps**

**Introduction**


**Theory**

The Self-Organising Feature Map (SOM) is a competitive learning algorithm proposed by Teuvo Kohonen in 1988.
The SOM feature map comprises of a grid of inter-connected neurons.

The SOM is perfectly topology preserving which means that if the dimensionality of the input and the feature map correspond, then the relative ordering of the inputs is preserved by ordering in the neurons, so that neurons that are close together represent inputs that are close together, while neurons that are far apart represent inputs that are far apart.

Algorithm Pseudocode:



1.   Initialisation
    *   Choose a size (number of neurons) and number of dimensions d for the map.
    *   Choose random values for the weight vectors so that they are all different.

2.   Learning

    – repeat:
        * for each datapoint:
            * Select the best-matching neuron nb using the minimum Euclidean distance
between the weights and the input,

            nb = minjkx − wTj k.
            * Update the weight vector of the best-matching node using:

            wTj  wTj + (t)(x − wTj ),

            where (t) is the learning rate.

            * update the weight vector of all other neurons using:

            wTj   wTj + n(t)h(nb, t)(x − wTj ),

            where n(t) is the learning rate for neighbourhood nodes, and h(nb, t) is the neighbourhood function, which decides whether each neuron should be included in the neighbourhood of the winning neuron (so h = 1 for neighbours and h = 0 for non-neighbours)

            * reduce the learning rates and adjust the neighbourhood function, typically by (t+1) = 
(t)k/kmax where 0  
  1 decides how fast the size decreases, k is the number of iterations the algorithm has been running for, and kmax
is when you want the learning to stop. The same equation is used for both learning rates (, n) and the neighbourhood function h(nb, t).

    – until the map stops changing or some maximum number of iterations is exceeded

3. Usage:

    – for each test point:
    * select the best-matching neuron nb using the minimum Euclidean distance between the weights and the input:

    nb = minjkx − wTj k

**Message Box**

Remaining- 
1. Reading csv data and standardization
2. Integration and Testing
3. Documentation
4. Deciding optimal lattice size and other discussions

Update in code 
1. Standardized the dataset 
2. Initially the lattice neurons were initialized randomly with the same number due to the same seed used to initialize all the neurons so I changed the code which will randomly generate weights for neuron with different seeds .
3. Now we can experiment with different lattice sizes along with learning rate , choosing the initial distribution for neurons and decay function of learning rate.


**Source Code**

In [11]:
%%writefile neuron.cpp
#include <iostream>
#include <vector>
#include <string>
#include <fstream>
#include <sstream>
#include <random>
#include <cstdlib>
#include <time.h>
using namespace std;

//1. Class Country_Stats to store the data about the countries
class Country_Stats
{
  public:

  string country; //Name of the country
  vector<double> stats; //stores the 9 attributes
  void print_country()
  {
      cout<< country <<" ";
      for(auto x:stats) cout<< x<<" ";
      cout<<endl;
  }
};

//2. Neurons are treated as vectors of number of dimensions equal to the number of features in the input data
class Neuron
{
    public:

    int weight_size; //equal to number of features
    vector<double>weights; //weights vector
    pair<int,int>coordinate; //location of neuron in lattice

    void set_neuron_coordinates(const int weight_size,int x_cord,int y_cord)
    {
        this->weight_size=weight_size;
        this->weights.resize(weight_size,0.0);
        this->coordinate.first=x_cord;
        this->coordinate.second=y_cord;
        return;
    }
    vector<double> Initialize_neuron_normal()
    {
        /*
        We don't need mean or std of the features since we are standardizing the data
        our feature values will follow standard normal distribution with mean=0 and std=1
        */
      
       random_device rd{};
       mt19937 gen{rd()};
 
        // values near the mean are the most likely
        // standard deviation affects the dispersion of generated values from the mean
        for (int i=0;i<this->weight_size;i++){
            normal_distribution<double>dist{0,1};
            weights[i]=dist(gen);
        }
        return weights;
    }
    //Randomly Initialize neuron weights
    void initialize_neuron_random(const int x,const int y)
    {
      unsigned seed = 10*(x+1)+y;
      srand(seed);
        
        for (int i=0;i<this->weight_size;i++)
        {
            
            double random_number_oto1= (double)(rand()/(1.0*RAND_MAX));
         //   cout<<random_number_oto1<<"Generated random number\n";
            //a random number from 0 to 1 is assigned to weights
            weights[i]=random_number_oto1;
        }
        return;
    }
};

//3. A Lattice class to organize the neurons. The SOM algorithm will operate on this
class Lattice
{
  public:

    int lattice_length;
    int lattice_width;
    vector<vector<Neuron>> lattice;
    //Constructors
    Lattice(){}
    Lattice(int length, int width)
    {   //set size of lattice if given
        lattice_length=length;
        lattice_width=width;
        lattice.resize(length,vector<Neuron>(width));
    }
    //Define positions of neurons and initialize their weights
    void initialize_Lattice_Neurons(const int number_of_features,string distribution="r")
   {
       for(int i=0;i<lattice_length;i++)
        for(int j=0;j<lattice_width;j++)
        {
          lattice[i][j].set_neuron_coordinates(number_of_features,i,j);//Set position and size of neurons
          if(distribution=="r"){
            lattice[i][j].initialize_neuron_random(i,j);//Initialize weights
          }
          else if(distribution=="n"){
            lattice[i][j].Initialize_neuron_normal();
          }
          else{
            cout<<"No distribution with that alphabet\n";
          }
         // 
          ////Initialize weights from normal distribution
        }
   }
   
};

//4. Contains all the Functions used to implement the SOM Algorithm
class Self_Organising_Map
{
  public:
  double learning_rate;//Name is self explanatory
  int number_of_epochs;//The number of Iterations for which the Algorithm will run
  vector<pair<Neuron,double>>winners;
  //Parameterized Constructor
  Self_Organising_Map(double lr, int num_epochs)
  {
    learning_rate=lr;
    number_of_epochs=num_epochs;
  }

  //a. Find Distance between a Neuron's weight vector and a Row vector of Data
  double find_neuron_distance(const Neuron& neuron, const Country_Stats &country_stats)
  {
    double ans=0.0;
    int number_of_features= neuron.weight_size;
    for(int i=0;i<number_of_features;i++)
      ans+= (country_stats.stats[i] -neuron.weights[i])*(country_stats.stats[i] -neuron.weights[i]);
    //Return the Euclidean distance
    return (double) sqrt(ans);
  }
  void PrintLattice(const Lattice *SOM)
  {
    int row=SOM->lattice_length;
    int cols=SOM->lattice_width;
    for(int i=0;i<row;i++){
        for(int j=0;j<cols;j++){
            int x_cord=SOM->lattice[i][j].coordinate.first;
            int y_cord=SOM->lattice[i][j].coordinate.second;
            cout<<"Coordinates of the neuron "<<x_cord<<" "<<y_cord<<"\n";
            for(auto it:SOM->lattice[i][j].weights){
                cout<<it<<" ";
            }
            cout<<"\n";
        }
    }
  }

  //b. Find the Winner Neuron for a given country
  pair<Neuron,double> find_winner_neuron(const Lattice * SOM, const Country_Stats &country_stats)
  {
    //Initialize winner neuron with the first neuron of lattice
    Neuron winner=SOM->lattice[0][0];
    double min_distance=find_neuron_distance(winner,country_stats);
    //For all neurons of the Lattice
    for(int i=0;i<SOM->lattice_length;i++)
      for(int j=0;j<SOM->lattice_width;j++)
      {
        double curr_neuron_distance=find_neuron_distance(SOM->lattice[i][j],country_stats);
        //If distance of current neuron is less than the winner till now
        if(curr_neuron_distance< min_distance)
        {
          winner=SOM->lattice[i][j];//Declare current neuron as winner
          min_distance=curr_neuron_distance;//Update the min distance with new winner disatnce
        }
      }
    return {winner,min_distance};
  }

  //c. Update the neurons' weights based on its distance(neighborhood) with the winner.
  // The more closer a neuron is to the winner, more its weights are affected
  void update_neuron_weights(Lattice* SOM, Neuron &winner, const Country_Stats &country_stats)
  {
    int winnerx=winner.coordinate.first;
    int winnery=winner.coordinate.second;
    int num_features=winner.weight_size;
    //For all neurons of the lattice
    for(int neuronx=0;neuronx < SOM->lattice_length;neuronx++)
        for(int neurony=0;neurony < SOM->lattice_width;neurony++)
        {
            //Calculate the distance between the neuron and the winner and the neighborhood function
            int d=(neuronx-winnerx)*(neuronx-winnerx) + (neurony-winnery)*(neurony-winnery);
            double neighbourhood_function=exp(-pow(d,0.5));
            //Update the neuron's weight vector
            for(int i=0;i<num_features;i++)
            {
                // wij = wij(old) + alpha(t) * neighbourhood_finction * (xik - wij(old))
                SOM->lattice[neuronx][neurony].weights[i]+= learning_rate*neighbourhood_function*(country_stats.stats[i]- SOM->lattice[neuronx][neurony].weights[i]);
            }
        }
  }
  
void preprocess_data(vector<Country_Stats>&country_data)
{
    int number_of_countries=country_data.size();
    int number_of_features=country_data[0].stats.size();
    vector<double>sum(number_of_features,0.0);
    vector<double>std(number_of_features,0.0);
    for(int i=0;i<number_of_countries;i++){
        for(int j=0;j<number_of_features;j++){
            sum[j]+=country_data[i].stats[j];
        }
    }
    for(int i=0;i<number_of_features;i++){
        sum[i]=sum[i]/number_of_countries;
    }
    for(int i=0;i<number_of_countries;i++){
        for(int j=0;j<number_of_features;j++){
            std[j]+=(country_data[i].stats[j]-sum[j])*(country_data[i].stats[j]-sum[j]);

        }
    }
    for(int i=0;i<number_of_features;i++){
        std[i]=std[i]/(number_of_countries-1);
    }
    for(int i=0;i<number_of_countries;i++){
        for(int j=0;j<number_of_features;j++){
            if(std[j]==0){
                continue;
            }
            country_data[i].stats[j]=(country_data[i].stats[j]-sum[j])/(std[j]);

        }
    }
    //return country_data;
    
}
  double error(int rows=1){
    double distance=0;
    for(auto it:winners){
      distance+=it.second;
    }
    distance=distance/rows;
    return distance;

  }

  //d. Change the learning rate based on the number of iterations done till now and the total number of epochs
  
  double change_learning_rate(const int epochs,const string method="d")//d for decay using formula 1 and l for decay using formula 2
  {
    if(method=="d"){
      return (double)((learning_rate*epochs)/(number_of_epochs*1.0));//formula 1
    }
    else if(method=="l"){
      return (double)((learning_rate*exp((-epochs)/(number_of_epochs*1.0))));//formula 2
    }
    else{
      cout<<"No suitable decay chosen hence learning rate not changed\n";
      return learning_rate;
    }
    
  }
  void PrintNeuronweights(const Neuron &neuron)
  {
    for(auto it:neuron.weights){
        cout<<it<<" ";
    }
    cout<<'\n';
  }
  //e. Integrate all the functionalities and train the Self Organizing Map
  void simulate_SOM(Lattice* SOM_lattice,  vector<Country_Stats> & country_data, const int number_of_features,const string dist="r")
  {
    //Initialize the Lattice of neurons
    cout<<"Starting Simulation\n";
    preprocess_data(country_data);
    SOM_lattice->initialize_Lattice_Neurons(number_of_features,dist);

    //PrintLattice(SOM_lattice);
    //Repeat for number of epochs (iterations)
    int rows=country_data.size();
    for(int epoch=1;epoch<=number_of_epochs;epoch++)
    {
      // for each country
      for(int i=0;i<rows;i++)
      {
        //Find the winner and update the weights of neurons
        pair<Neuron,double>p= find_winner_neuron(SOM_lattice,country_data[i]);
        //cout<<"Initial weights of winner\n";
        //PrintNeuronweights(SOM_lattice->lattice[winner.coordinate.first][winner.coordinate.second]);
        winners.push_back(p);
        update_neuron_weights(SOM_lattice,p.first,country_data[i]);
        //cout<<"Updated weights of winner\n";
        //PrintNeuronweights(SOM_lattice->lattice[winner.coordinate.first][winner.coordinate.second]);
      }
      //Change the learning rate after each epoch
      double loss=error(rows);
      learning_rate=change_learning_rate(epoch,"l");
      cout<<"Current Epoch number #"<<epoch<<" Error :"<<loss<<" Learning rate changed to ->"<<learning_rate<<"\n";
      winners.clear();
    }
   // cout<<"Final lattice weights\n";
    //PrintLattice(SOM_lattice);
  }
};

int main()
{
    vector<Country_Stats> country_data;
     int number_of_features=9;
    // **Reading the data**

    fstream fin;
    fin.open("Country-data-refined.csv", ios::in);
    //Temporary variables to help in reading the file
    vector<string> row;
    string line, word;
    Country_Stats temp_country;
    //Consume the first row of headers
    getline(fin, line);

    while (getline(fin, line))
    {   //clear the row variable and store new row of data in it
        row.clear();
        stringstream s(line);
        while (getline(s, word, ','))
          row.push_back(word);
        //store data in temp_country from row variable
        temp_country.country= row[0];
        temp_country.stats.clear();
        for(int i=1;i<row.size();i++) temp_country.stats.push_back(stod(row[i]));
        //push this temp_country in country_data
        country_data.push_back(temp_country);
    }

    //**Action Time**

   int proposed_lattice_length=4;
   int proposed_lattice_width=4;
   //Instantiating
   //preprocess_data(country_data);
   Lattice *SOM_lattice= new Lattice(proposed_lattice_length,proposed_lattice_width);
   Self_Organising_Map *SOM=new Self_Organising_Map(0.5,10);
   //Train the Self Organizing Map
   SOM->simulate_SOM(SOM_lattice,country_data,number_of_features,"n");
   Country_Stats worst_country;
   worst_country.country="Worst Country";
   vector<double>worst_stats{1,0,0,1,0,1,0,0,0};
   worst_country.stats=worst_stats;

   //Results

   pair<Neuron,double>temp = SOM->find_winner_neuron(SOM_lattice,worst_country);
   cout<<"Cluster coordinate of worst countries "<<temp.first.coordinate.first<<" "<<temp.first.coordinate.second<<"\n";
   int coord_x_worst=temp.first.coordinate.first;
   int coord_y_worst=temp.first.coordinate.second;
   vector<string>worst_country_name;
   for(auto it:country_data){
    pair<Neuron,double>temp = SOM->find_winner_neuron(SOM_lattice,it);
    if(temp.first.coordinate.first==coord_x_worst && temp.first.coordinate.second==coord_y_worst){
      worst_country_name.push_back(it.country);
    }

   }
   //SOM_lattice->initialize_Lattice_Neurons(10);
   //SOM->PrintNeuronweights(SOM_lattice->lattice[0][0]);
   
    cout << "Hello world!" << endl;
    for(auto it:worst_country_name){
      cout<<it<<"\n";
    }
    return 0;
}


Overwriting neuron.cpp


**Command to run the source code**

In [12]:
%%script bash

g++ neuron.cpp 
./a.out

Starting Simulation
Current Epoch number #1 Error :0.367956 Learning rate changed to ->0.452419
Current Epoch number #2 Error :0.294358 Learning rate changed to ->0.370409
Current Epoch number #3 Error :0.29312 Learning rate changed to ->0.274406
Current Epoch number #4 Error :0.291221 Learning rate changed to ->0.18394
Current Epoch number #5 Error :0.286555 Learning rate changed to ->0.111565
Current Epoch number #6 Error :0.279868 Learning rate changed to ->0.0612282
Current Epoch number #7 Error :0.274548 Learning rate changed to ->0.030405
Current Epoch number #8 Error :0.271464 Learning rate changed to ->0.0136619
Current Epoch number #9 Error :0.269486 Learning rate changed to ->0.0055545
Current Epoch number #10 Error :0.268515 Learning rate changed to ->0.00204339
Cluster coordinate of worst countries 1 2
Hello world!
Cambodia
Dominican Republic
Paraguay


Changes made :
1. I made a function for changing learning rate to change learning rate per epoch
2. Created a new function for updating the weights of winner and neighbourhood neuron taken from the paper I was refering to yesterday .
3. Practically I think we should initialize our lattice neurons from a particular distribution like normal or uniform but we can leave that for experimentation
   
   Yes, we'll do that. I had done that randomly just for testing the code as mean and sd vectors were not available.

Features of our Self Organizing Maps are 
1. Leaky Learning all the neighbours of winner neuron is updated with variable amounts
2. Changing Learning Rate with increase in epochs


Finding the country which needs the most help 

The country which would be in dire need of help will have </br>
high child mortality(1) </br>
very less exports(0) </br>
very poor health spending(0) </br>
Very much dependent on other countries hence very high import(1)  </br>
Low income (0)</br>
High inflation(1) </br>
Low life_expectancy(0) </br>
Due to poor health funding the fertility rate of women will also be very low(0) </br>
Very Low GDP(0) </br>
Our stats for the most help needed country would be {1,0,0,1,0,1,0,0,0}
Using this info we find the winner neuron for this country i.e the cluster coordinate of such countries
Then find the winner neuron for the entire country data and find the country which belong to this cluster





Final Results 
The top 3 countries where HELP NGO should focus on are in no particular order
1. Cambodia
2. Dominican Republic
3. Paraguay

**References**